In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#%matplotlib tk

In [2]:
result = pd.read_csv("/home/lin/git/summerProject/Heston++onSPXonly_ana_estar15_100run")

In [3]:
result[['time', 'iteration', 'pv', 'jac', 'lin_sys', 'e0', 'e*', 'Je', 'Dp', 'count','count2', 'count7']].describe()

,time,iteration,pv,jac,lin_sys,e0,e*,Je,Dp,count,count2,count7
count,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,100.000000,100.0,100.000000
mean,2.620456,277.520000,305.440000,277.600000,304.440000,2.517316e+05,1.316055e+04,1.993225e+05,2.401137e-06,1.330000,0.0,0.330000
std,1.710542,173.105239,201.611504,173.047375,201.611504,1.472470e+06,9.382441e+04,1.512421e+06,1.566389e-05,0.841535,0.0,0.841535
min,0.406752,44.000000,45.000000,44.000000,44.000000,9.874506e-04,4.667708e-19,1.520071e-12,0.000000e+00,1.000000,0.0,0.000000
25%,1.079037,114.750000,129.750000,114.750000,128.750000,3.921170e-02,3.492491e-16,2.918677e-07,9.356099e-09,1.000000,0.0,0.000000
50%,1.978325,228.500000,238.000000,228.500000,237.000000,1.339994e-01,9.353200e-16,8.190531e-07,3.423440e-07,1.000000,0.0,0.000000
75%,4.394372,500.000000,501.000000,500.000000,500.000000,2.748652e-01,8.671862e-13,6.451634e-06,7.656390e-07,1.000000,0.0,0.000000
max,6.087996,500.000000,704.000000,500.000000,703.000000,1.211478e+07,8.413986e+05,1.423435e+07,1.559902e-04,6.000000,0.0,5.000000


In [4]:
result['stopped_by'].unique()
#This show the reasons for stopping:
#1 - stopped by small gradient J^T e
#2 - stopped by small Dp
#3 - stopped by itmax
#4 - singular matrix. Restart from current p with increased \mu
#5 - no further error reduction is possible. Restart with increased mu
#6 - stopped by small ||e||_2
#7 - stopped by invalid (i.e. NaN or Inf) "func" values; a user error

array([6, 3, 1, 5])

In [5]:
#number of solved cases, allowing restart when stopped by 7 or 2
solved = result[result['stopped_by'] == 6]
solved.count().unique()

array([61])

In [6]:
#percentage of solved
solved.count().unique()/result.count().unique()

array([0.61])

In [7]:
#number of solved cases at the first try
solved_first_time = solved[solved['count'] == 1]
solved_first_time.count().unique()

array([50])

In [8]:
#number of cases stopped by small Dp (update of parameter)
DP = result[result['stopped_by'] == 2]
DP.count().unique()

array([0])

In [9]:
#number of cases that stopped by max iteration
itmax = result[result['stopped_by'] == 3]
itmax.count().unique()

array([28])

In [10]:
#number of cases that stopped by mall gradient J^T e
smalJac = result[result['stopped_by'] == 1]
smalJac.count().unique()

array([8])

In [11]:
#average time for all solved cases
solved['time'].describe()['mean']

1.7985538032786885

In [12]:
#average time for cases solved without restart
solved_first_time['time'].describe()['mean']

1.7630635

In [13]:
#Average number of iteration for solved cases
solved_first_time['iteration'].describe()['mean']

193.28

In [14]:
#Average number of iteration for all solved cases
solved['iteration'].describe()['mean']

198.0327868852459

In [15]:
#All solved cases
solved[['Ok', 'Rk', 'Ovbar', 'Rvbar','Ov0', 'Rv0', 'Orho', 'Rrho', 'Osigma', 'Rsigma', 'OI1', 'RI1', 'OI2', 'RI2', 'stopped_by', 'e*']]

,Ok,Rk,Ovbar,Rvbar,Ov0,Rv0,Orho,Rrho,Osigma,Rsigma,OI1,RI1,OI2,RI2,stopped_by,e*
0,1.091935,1.091920,0.462782,0.462785,0.247063,0.247063,-0.643092,-0.643092,0.891224,0.891224,0.000519,0.000519,0.000035,0.000035,6,1.403446e-16
2,2.228506,2.228640,0.872144,0.872436,0.467802,0.468001,-0.140097,-0.140067,0.743349,0.743184,0.000134,0.000125,0.000698,0.000688,6,4.282626e-16
3,4.514202,4.508319,0.805474,0.807836,0.240793,0.241476,-0.204607,-0.204342,0.297576,0.297129,0.000449,0.000414,0.000758,0.000710,6,9.328751e-16
7,1.450034,1.460961,0.392033,0.399941,0.906963,0.902791,-0.314628,-0.315372,0.304921,0.305632,0.000632,0.000784,0.000169,0.000282,6,9.643388e-16
8,0.958333,0.957368,0.621978,0.621246,0.765146,0.765293,-0.702454,-0.702352,0.620122,0.620087,0.000603,0.000598,0.000321,0.000319,6,6.889399e-16
9,1.919512,1.922033,0.525241,0.525693,0.579613,0.579307,-0.444557,-0.444678,0.383097,0.383204,0.000436,0.000447,0.000381,0.000388,6,7.841706e-16
11,2.177440,2.173594,0.328619,0.329152,0.143186,0.143418,-0.721732,-0.720704,0.099769,0.099751,0.000170,0.000160,0.000307,0.000297,6,9.959554e-16
13,3.594618,3.593365,0.801914,0.802586,0.924771,0.926012,-0.164391,-0.164282,0.743028,0.742494,0.000298,0.000250,0.000812,0.000769,6,7.512144e-16
14,1.247102,1.246674,0.194122,0.193782,0.645663,0.645794,-0.859521,-0.859326,0.608930,0.608941,0.000023,0.000018,0.000725,0.000722,6,5.481412e-16
17,3.136803,3.137071,0.073694,0.073706,0.735943,0.736109,-0.161878,-0.161859,0.883048,0.882953,0.000436,0.000430,0.000836,0.000831,6,7.342933e-16


In [16]:
#next 7 blocks show the average distance between initial guess and real parameter
disIk = np.abs(solved['Ik'] - solved['Rk']).mean()
disIk

1.1757920862723674

In [17]:
disIvbar = np.abs(solved['Ivbar'] - solved['Rvbar']).mean()
disIvbar

0.2684872777950839

In [18]:
disIv0 = np.abs(solved['Iv0'] - solved['Rv0']).mean()
disIv0

0.2608432022464965

In [19]:
disIsigma = np.abs(solved['Isigma'] - solved['Rsigma']).mean()
disIsigma

0.29376722885605033

In [20]:
disIrho = np.abs(solved['Irho'] - solved['Rrho']).mean()
disIrho

0.25911951876417116

In [21]:
disII1 = np.abs(solved['II1'] - solved['RI1']).mean()
disII1

0.00036305040969208377

In [22]:
disII2 = np.abs(solved['II2'] - solved['RI2']).mean()
disII2

0.00031009655043543177

In [23]:
#next 7 blocks show the average distance between optimal found and real parameter
disOk = np.abs(solved['Ok'] - solved['Rk']).mean()
disOk

0.0035757197794941058

In [24]:
disOvbar = np.abs(solved['Ovbar'] - solved['Rvbar']).mean()
disOvbar

0.0010780179481016923

In [25]:
disOsigma = np.abs(solved['Osigma'] - solved['Rsigma']).mean()
disOsigma

0.04531444425785349

In [26]:
disOrho = np.abs(solved['Orho'] - solved['Rrho']).mean()
disOrho

0.016047778906912637

In [27]:
disOv0 = np.abs(solved['Ov0'] - solved['Rv0']).mean()
disOv0

0.001289194277595836

In [28]:
disOI1 = np.abs(solved['OI1'] - solved['RI1']).mean()
disOI1

5.044731611947653e-05

In [29]:
disOI2 = np.abs(solved['OI2'] - solved['RI2']).mean()
disOI2

4.6242136822970816e-05